In [4]:
!pip install neuron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from neuron import h
from neuron.units import mV, ms, um,mm
h.load_file("stdrun.hoc")

import plotly
import plotly.graph_objects as go

# Defineeate a section (soma) of the neuron
soma = h.Section(name="soma")
soma.L = soma.diam = 10 * um

# Insert properties of the HH model
h.hh.insert(soma)

# Define Single Electrode Clamp
vclamp = h.SEClamp(soma(0.5))
vclamp.amp1 = -65 * mV
vclamp.dur1 = 5 * ms
vclamp.dur2 = 11 * ms
vclamp.amp3 = -65 * mV
vclamp.dur3 = 10000 * ms

# Record time and ion currents
t = h.Vector().record(h._ref_t)
ina = h.Vector().record(soma(0.5)._ref_ina)
ik = h.Vector().record(soma(0.5)._ref_ik)

In [6]:
k_fig = go.Figure()
na_fig = go.Figure()

# Simulate for sodium at various voltages
for v in [-109,-100,-88,-76,-63,-51,-38,-32,-26,-19,-10,-6]:
    vclamp.amp2 = v+120
    h.finitialize(-65 * mV)
    h.continuerun(20 * ms)
    # Calculate conductance from current and voltage
    na_fig.add_trace(go.Scatter(x=t, y=ina/(v+120), name=f"v={v}", line={"width": 4}))
    
# Simulate for potassium at various voltages
for v in [-109,-100,-88,-76,-63,-51,-38,-32,-26,-19,-10,-6]:
    vclamp.amp2 = v+142
    h.finitialize(-65 * mV)
    h.continuerun(20 * ms)
    # Calculate conductance from current and voltage
    k_fig.add_trace(go.Scatter(x=t, y=(ik/(v+142)), name=f"v={v}", line={"width": 4}))

# Plot for potassium
k_fig.update_layout(dict(
    xaxis_title="t (ms)",
    yaxis_title="gk (m.mho/cm^2)",
    title="Voltage clamp experiments: Potassium"
))

# Plot for sodium
na_fig.update_layout(dict(
    xaxis_title="t (ms)",
    yaxis_title="gna (m.mho/cm^2)",
    title="Voltage clamp experiments: Sodium"
))

k_fig.show()
na_fig.show()